## day2greg

This notebook implements the methodology dev'd by Jonny Williams to adjust 365 day and 360 day calendar climate datasets to standard gregorian. It improves on the original notebook by standardizing the routine, leveraging numpy's array functions for fast vectorized operations, and using far fewer dependencies.

See 
- https://github.com/jonnyhtw/360day2greg/tree/main ; and
- https://eartharxiv.org/repository/view/5645/

In [1]:
from netCDF4 import Dataset
import os
import sys
import numpy as np
import calendar

os.chdir(r'C:\Projects\Peel_ClimateChange\Data_Downloads') #just my working dir

## Just two functions!

One to get an array of indices where to perform the interpolated insertions, and another to do them.

In [23]:
def getGregorianSeeds(num_yrs,y0,day_cal):
    ''' used to obtain array of random "seeds" for adding extra interpolated days within each year.
    args:
    num_yrs = total number of years
    y0 = year zero (e.g. 1999)
    day_cal = the number of days in the calendar you're using (e.g. 360 or 365)
    returns: 
        list of indices at which to perform "interpolated insertions"
    '''
    assert day_cal <= 365, 'getSeeds only configured for adding interpolated days, day_cal must be 365 or less'
    ed = (365 - day_cal) # how many extra days you need to add to a "normal" year
    ins = [] #running tally of indices to flag where interpolation should occur
    #starting in y0 for the spec'd number of years...
    for i,y in enumerate(range(y0,y0+ny,1)):
        #check if leap year
        if calendar.isleap(y):
            b = ed+2 #b = number of divisions to apply to year
        else:
            b = ed+1
        sr = np.linspace(0,day_cal,b) #makes "bins" within the year by which to sample random seeds
        if len(sr) == 1: #special case just handling if you don't need to sample any extra days
            s = day_cal
        else:
            s = sr[1]-sr[0] #size of each bin 
        rs = np.round(np.random.rand(b-1)*s) #makes b-1 random numbers between 0 and 1 multiplied by the bin size to get an "offset" from sr
        seeds = (sr[:-1]+rs).astype(int) #adds the offsets to the divisions 
        ins.extend(seeds+day_cal*i) #extends the running tally by the seeds + the number of days elapsed up to the current year
    return ins

def getInterpolatedData(arr,ins):
    '''Inserts interpolated data into existing array given list of indices where to insert. DOES NOT ALTER EXISTING ARRAY
    If indice is in ins, an additional new value will be inserted set to "halfway" between the current value and the next value.  
    args:
        - arr: array of data to get extra days inserted to it, typically for transient grid will be of shape (time,col,row)
        - ins: list or array of of indices at which to do the interpolated insertion
    returns: new array of length len(arr) + len(ins) containing the interpolated values in the spec'd indices
        '''
    r = [] #new data
    for i,a in enumerate(va):
        r.append(a)
        if i in ins:

            r.append((va[i]+va[i+1])/2)
    return np.array(r)

# example usage 1

- UKESM1 model is 360 day

In [24]:
#load net cdf of precip
fn = r'Precip\\pr_day_bccaqv2_anusplin300_ukesm1_0_ll_historical_ssp585_r1i1p1f2_gn_19500101_21001230_sub.nc'
d = Dataset(fn)
print('dataset calendar:')
print(d.variables['time'].calendar) #360 day.

va = d.variables['pr'][:].transpose((2,0,1)) #transposes variable from shape (col,row,time) into array (time,col,row)
ny = int(va.shape[0]/360) #number of years
y0 = 1950 #starting year

ins = getGregorianSeeds(num_yrs=ny,y0=1950,day_cal=360) #gets a list of indices in the entire dataset where to do the interpolated insertions
# print(ins)
out = getInterpolatedData(arr=va,ins=ins) #makes a new array with the interpolated arrays inserted

#check
print('number of original days:')
print(len(va))
print('number of inserted days:')
print(len(ins))
print('number of gregorian-based days from y0 for ny')
import pandas as pd #using this only for convenience of checking dates
print(len(pd.date_range(start='Jan 1, 1950',end='Dec 31 2100',freq='1D')))
print('number of days in updated dataset:')
print(out.shape[0])
del d 


dataset calendar:
360_day
number of original days:
54360
number of inserted days:
792
number of gregorian-based days from y0 for ny
55152
number of days in updated dataset:
55152


# example usage 2

- CMCC-ESM2 model is 365 day, want to insert an extra day of interpolated values randomly within each leap year

In [22]:
#load net cdf of precip
fn = r'Precip\\pr_day_bccaqv2_anusplin300_cmcc_esm2_historical_ssp585_r1i1p1f1_gn_19500101_21001231_sub.nc'
d = Dataset(fn)
print('dataset calendar:')
print(d.variables['time'].calendar) #365 day.

va = d.variables['pr'][:].transpose((2,0,1)) #transposes variable from shape (col,row,time) into array (time,col,row)
ny = int(va.shape[0]/365) #number of years
y0 = 1950 #starting year

ins = getGregorianSeeds(num_yrs=ny,y0=1950,day_cal=365) #gets a list of indices in the entire dataset where to do the interpolated insertions
out = getInterpolatedData(arr=va,ins=ins) #makes a new array with the interpolated arrays inserted

#check
print('number of original days:')
print(len(va))
print('number of inserted days:')
print(len(ins))
print('number of gregorian-based days from y0 for ny')
import pandas as pd #using this only for convenience of checking dates
print(len(pd.date_range(start='Jan 1, 1950',end='Dec 31 2100',freq='1D')))
print('number of days in updated dataset:')
print(out.shape[0])

del d

dataset calendar:
365_day
number of original days:
55115
number of inserted days:
37
number of gregorian-based days from y0 for ny
55152
number of days in updated dataset:
55152
